In [ ]:
# -*- coding: utf-8 -*-
"""
多模式多波长光场调制系统 - 主程序（修改版）
集成训练-仿真工作流程 - 无需额外模块版本
"""

import torch
import numpy as np
import matplotlib.pyplot as plt
import os
import time
from datetime import datetime

# 导入自定义模块
from label_utils import create_evaluation_regions_mode_wavelength, evaluate_output, evaluate_all_regions, visualize_labels
from config import Config
from data_generator import MultiModeMultiWavelengthDataGenerator
from visualizer import Visualizer
from trainer import Trainer
from model import MultiModeMultiWavelengthModel
from simulator import Simulator

# 记录开始时间
start_time = time.time()

# 设置随机种子，确保结果可重现
torch.manual_seed(42)
np.random.seed(42)

print("=" * 60)
print("多模式多波长光场调制系统 - 训练-仿真集成版")
print("=" * 60)

# ===== 掩码加载器类（内联定义）=====
class SimpleMaskLoader:
    """简化的相位掩码加载器"""
    
    def __init__(self, config):
        self.config = config
    
    def create_fallback_masks(self, num_layers=3):
        """创建备用聚焦掩码"""
        print("⚠ 创建备用聚焦掩码...")
        
        def create_focusing_mask(size, wavelength, focal_length, pixel_size):
            center = size // 2
            y, x = np.ogrid[:size, :size]
            r_squared = ((x - center) * pixel_size) ** 2 + ((y - center) * pixel_size) ** 2
            k = 2 * np.pi / wavelength
            phase = -k * r_squared / (2 * focal_length)
            return np.mod(phase, 2 * np.pi)
        
        masks = []
        focal_lengths = [50e-3, 100e-3, 150e-3]  # 不同层的焦距
        
        for layer_idx in range(num_layers):
            layer_masks = []
            for wl_idx, wavelength in enumerate(self.config.wavelengths):
                focal_length = focal_lengths[layer_idx % len(focal_lengths)]
                mask = create_focusing_mask(
                    self.config.layer_size, wavelength, focal_length, self.config.pixel_size
                )
                layer_masks.append(mask)
            masks.append(layer_masks)
        
        print(f"✓ 创建了 {len(masks)} 层备用掩码")
        return masks
    
    def get_masks_for_simulation(self, trained_masks=None, num_layers=3):
        """获取用于仿真的掩码"""
        if trained_masks is not None:
            print("✓ 使用训练好的相位掩码进行仿真")
            return trained_masks
        else:
            print("⚠ 使用备用掩码进行仿真")
            return self.create_fallback_masks(num_layers)

# 创建配置
config = Config(
    # 基本参数
    num_modes=3,                                # 模式数量
    wavelengths=np.array([450e-9, 550e-9, 650e-9]),  # 波长列表(m)
    
    # 空间参数
    field_size=50,                              # 场大小(像素)
    layer_size=200,                             # 层大小(像素)
    focus_radius=5,                             # 焦点半径(像素)
    detectsize=15,                              # 检测区域大小(像素)
    
    # 物理参数
    z_layers=40e-6,                             # 层间距离(m)
    z_prop=300e-6,                              # 传播距离(m)
    z_step=20e-6,                               # 传播步长(m)
    pixel_size=1e-6,                            # 像素大小(m)
    
    # 检测区域偏移 - 为每个波长定义不同的偏移
    offsets=[(0,0), (20,0), (-20,0)],           # 每个波长的检测区域偏移
    
    # 训练参数
    learning_rate=0.01,                         # 学习率
    lr_decay=0.99,                              # 学习率衰减
    epochs=700,                                 # 训练轮数
    batch_size=32,                               # 批量大小
    
    # 保存参数
    save_dir="./results_multi_mode_multi_wl/",  # 保存目录
    flag_savemat=True                           # 是否保存.mat文件
)

# 确保保存目录存在
os.makedirs(config.save_dir, exist_ok=True)
os.makedirs(os.path.join(config.save_dir, "trained_models"), exist_ok=True)

# ===== 阶段1: 数据准备 =====
print("\n" + "="*50)
print("阶段1: 数据准备")
print("="*50)

# 创建数据生成器
print("创建数据生成器...")
data_generator = MultiModeMultiWavelengthDataGenerator(config)

# 生成多模式多波长标签
print("生成标签...")
labels = data_generator.generate_labels()

# 可视化标签布局
print("可视化标签布局...")
visualize_labels(labels, config.wavelengths)

# 创建评估区域
print("创建评估区域...")
evaluation_regions = create_evaluation_regions_mode_wavelength(
    config.layer_size, 
    config.layer_size, 
    config.focus_radius, 
    detectsize=config.detectsize
)

print("✓ 数据准备完成")

# ===== 阶段2: 模型训练 =====
print("\n" + "="*50)
print("阶段2: 模型训练")
print("="*50)

# 检查是否存在已训练的模型
trained_models_dir = os.path.join(config.save_dir, "trained_models")
existing_models = []
if os.path.exists(trained_models_dir):
    model_files = [f for f in os.listdir(trained_models_dir) if f.startswith("model_") and f.endswith("layers.pth")]
    existing_models = [f for f in model_files]

# 定义要训练的层数选项
num_layer_options = [1, 2, 3, 4, 5]  # 可以根据需要调整层数

# 询问是否使用现有模型或重新训练
use_existing = False
if existing_models:
    print(f"发现已存在的训练模型: {existing_models}")
    try:
        response = input("是否使用现有模型？(y/n，默认n): ").lower().strip()
        use_existing = response == 'y'
    except:
        print("使用默认选项：重新训练")
        use_existing = False

if use_existing and existing_models:
    print("加载现有训练模型...")
    results = {'models': [], 'losses': [], 'phase_masks': [], 'weights_pred': [], 'visibility': []}
    
    for num_layers in num_layer_options:
        model_path = os.path.join(trained_models_dir, f"model_{num_layers}layers.pth")
        if os.path.exists(model_path):
            print(f"加载 {num_layers} 层模型...")
            
            try:
                # 加载模型检查点
                checkpoint = torch.load(model_path, map_location='cpu')
                
                # 创建模型实例
                model = MultiModeMultiWavelengthModel(config, num_layers)
                model.load_state_dict(checkpoint['model_state_dict'])
                
                # 提取相位掩码
                phase_masks = []
                if hasattr(model, 'get_phase_masks_for_simulation'):
                    phase_masks = model.get_phase_masks_for_simulation()
                else:
                    # 兼容旧版本
                    for layer in model.layers:
                        phase = layer.phase.detach().cpu().numpy()
                        phase = phase % (2 * np.pi)
                        wavelength_masks = []
                        for _ in range(len(config.wavelengths)):
                            wavelength_masks.append(phase)
                        phase_masks.append(wavelength_masks)
                
                # 获取训练损失和可见度
                losses = checkpoint.get('train_losses', [])
                visibility = checkpoint.get('visibility', [])
                
                # 如果没有可见度数据，需要重新评估
                if not visibility:
                    print(f"  重新评估 {num_layers} 层模型...")
                    trainer_temp = Trainer(config, data_generator, MultiModeMultiWavelengthModel, evaluation_regions=evaluation_regions)
                    test_loader = trainer_temp._create_data_loaders()[1]
                    eval_results = trainer_temp._evaluate_model(model, test_loader)
                    visibility = eval_results['visibility']
                    weights_pred = eval_results['weights_pred']
                else:
                    weights_pred = []
                
                results['models'].append(model)
                results['losses'].append(losses)
                results['phase_masks'].append(phase_masks)
                results['weights_pred'].append(weights_pred)
                results['visibility'].append(visibility)
                
                print(f"✓ 成功加载 {num_layers} 层模型")
                
            except Exception as e:
                print(f"✗ 加载 {num_layers} 层模型失败: {e}")
                # 添加空结果以保持索引一致
                results['models'].append(None)
                results['losses'].append([])
                results['phase_masks'].append([])
                results['weights_pred'].append([])
                results['visibility'].append([])
        else:
            print(f"✗ 未找到 {num_layers} 层模型文件")
            # 添加空结果以保持索引一致
            results['models'].append(None)
            results['losses'].append([])
            results['phase_masks'].append([])
            results['weights_pred'].append([])
            results['visibility'].append([])
else:
    print("开始训练新模型...")
    
    # 创建训练器
    trainer = Trainer(config, data_generator, MultiModeMultiWavelengthModel, evaluation_regions=evaluation_regions)
    
    # 训练多个层数的模型
    results = trainer.train_multiple_models(num_layer_options)

print("✓ 模型准备完成")

# ===== 阶段3: 结果分析 =====
print("\n" + "="*50)
print("阶段3: 结果分析")
print("="*50)

# 检查可见度数据结构
print("检查训练结果...")
print(f"results键: {list(results.keys())}")
print(f"可见度数据结构: {len(results['visibility'])}层")

valid_results = []
for i, vis_data in enumerate(results['visibility']):
    if vis_data:  # 只处理非空的可见度数据
        expected_length = len(config.wavelengths) * config.num_modes
        print(f"第{i+1}层 ({num_layer_options[i]}层模型):")
        print(f"  数据长度: {len(vis_data)}")
        print(f"  期望长度: {expected_length} ({len(config.wavelengths)}波长 × {config.num_modes}模式)")
        
        if len(vis_data) == expected_length:
            print(f"  ✅ 数据长度匹配！")
            valid_results.append(i)
            # 按波长和模式重新组织显示
            vis_array = np.array(vis_data).reshape(len(config.wavelengths), config.num_modes)
            for wl_idx, wl in enumerate(config.wavelengths):
                wl_nm = wl * 1e9
                print(f"    {wl_nm:.0f}nm: 模式1={vis_array[wl_idx, 0]:.6f}, 模式2={vis_array[wl_idx, 1]:.6f}, 模式3={vis_array[wl_idx, 2]:.6f}")
        else:
            print(f"  ❌ 数据长度不匹配！")
    else:
        print(f"第{i+1}层 ({num_layer_options[i]}层模型): 无可见度数据")

# 可视化训练损失
if results['losses'] and any(results['losses']):
    print("可视化训练损失...")
    plt.figure(figsize=(10, 6))
    for i, num_layers in enumerate(num_layer_options):
        if results['losses'][i]:  # 确保有损失数据
            plt.plot(results['losses'][i], label=f'{num_layers} 层')
    plt.xlabel('训练轮次')
    plt.ylabel('损失值')
    plt.title('不同层数模型的训练损失')
    plt.legend()
    plt.grid(True)
    plt.savefig(f"{config.save_dir}/training_losses.png", dpi=300)
    plt.show()

# 可视化多波长可见度结果
if valid_results:
    print("可视化多波长可见度结果...")
    fig, axes = plt.subplots(1, len(config.wavelengths), figsize=(15, 5))
    if len(config.wavelengths) == 1:
        axes = [axes]

    wavelength_names = [f"{wl*1e9:.0f}nm" for wl in config.wavelengths]
    colors = ['red', 'green', 'blue']

    for wl_idx, (ax, wl_name) in enumerate(zip(axes, wavelength_names)):
        # 提取每个波长下不同层数模型的可见度
        layer_visibilities = []
        valid_layer_options = []
        
        for layer_idx in valid_results:
            vis_data = results['visibility'][layer_idx]
            if len(vis_data) == len(config.wavelengths) * config.num_modes:
                vis_array = np.array(vis_data).reshape(len(config.wavelengths), config.num_modes)
                wl_vis = vis_array[wl_idx, :]  # 该波长下所有模式的可见度
                layer_visibilities.append(wl_vis)
                valid_layer_options.append(num_layer_options[layer_idx])
        
        if layer_visibilities:
            layer_visibilities = np.array(layer_visibilities)
            
            # 绘制每个模式的可见度
            for mode_idx in range(config.num_modes):
                ax.plot(valid_layer_options, layer_visibilities[:, mode_idx], 
                        'o-', color=colors[mode_idx], label=f'模式 {mode_idx+1}')
        
        ax.set_xlabel('层数')
        ax.set_ylabel('可见度')
        ax.set_title(f'{wl_name} 可见度')
        ax.legend()
        ax.grid(True)
        ax.set_ylim(0, 1)

    plt.tight_layout()
    plt.savefig(f"{config.save_dir}/multi_wavelength_visibility.png", dpi=300)
    plt.show()

# 创建可视化器并生成详细分析
if valid_results:
    print("创建详细可视化分析...")
    try:
        visualizer = Visualizer(config)
        
        # 组织数据
        visibility_by_mode = visualizer.organize_visibility_by_mode(results, config, num_layer_options)
        
        # 绘制图表
        visualizer.plot_visibility_by_mode(visibility_by_mode, num_layer_options, 
                                           save_path=f"{config.save_dir}/visibility_by_mode.png")
        
        visualizer.plot_visibility_comparison_by_mode_wavelength(visibility_by_mode, num_layer_options,
                                                                save_path=f"{config.save_dir}/visibility_matrix.png")
        
        # 打印摘要和保存数据
        visualizer.print_visibility_summary(visibility_by_mode, num_layer_options)
        visualizer.save_visibility_data(visibility_by_mode, num_layer_options, f"{config.save_dir}/visibility_data.csv")
    except Exception as e:
        print(f"可视化分析出错: {e}")

print("✓ 结果分析完成")




In [ ]:
# ===== 阶段5: 保存最终结果 =====
print("\n" + "="*50)
print("阶段5: 保存最终结果")
print("="*50)

# 保存完整结果
print("保存完整结果...")
timestamp = datetime.now().strftime("%Y%m%d_%H%M")

# 准备保存数据
save_data = {
    'config': config.__dict__,  # 保存配置字典
    'models_state_dict': [model.state_dict() if model else {} for model in results['models']],
    'losses': results['losses'],
    'visibility': results['visibility'],
    'num_layer_options': num_layer_options,
    'timestamp': timestamp,
    'training_completed': True
}

# 如果有最佳模型，添加最佳模型信息
if 'best_idx' in locals():
    save_data.update({
        'best_model_idx': best_idx,
        'best_num_layers': best_num_layers,
        'best_avg_visibility': np.mean(best_visibility) if best_visibility else 0
    })

# 保存主结果文件
main_results_path = f"{config.save_dir}/complete_results_{timestamp}.pth"
torch.save(save_data, main_results_path)
print(f"✓ 主结果已保存: {main_results_path}")

# 保存单独的模型文件（便于后续加载）
for i, (model, num_layers) in enumerate(zip(results['models'], num_layer_options)):
    if model is not None:
        model_save_path = os.path.join(config.save_dir, "trained_models", f"model_{num_layers}layers.pth")
        torch.save({
            'model_state_dict': model.state_dict(),
            'model_config': {
                'num_layers': num_layers,
                'model_class': 'MultiModeMultiWavelengthModel'
            },
            'train_losses': results['losses'][i] if i < len(results['losses']) else [],
            'visibility': results['visibility'][i] if i < len(results['visibility']) else [],
            'config': config.__dict__,
            'timestamp': timestamp
        }, model_save_path)
        print(f"✓ {num_layers}层模型已保存: {model_save_path}")


In [ ]:
# ===== 阶段4: 光场传播仿真（所有模型）=====
print("\n" + "="*50)
print("阶段4: 光场传播仿真（所有模型）")
print("="*50)

# 获取所有有效模型
print("获取所有有效模型...")
valid_models = [(i, results['models'][i], results['visibility'][i], num_layer_options[i]) 
                for i in range(len(results['models'])) 
                if results['models'][i] is not None and results['visibility'][i]]

if valid_models:
    print(f"发现 {len(valid_models)} 个有效模型，将分别进行传播仿真")
    
    # 创建简化的掩码加载器
    print("准备仿真掩码...")
    mask_loader = SimpleMaskLoader(config)
    
    # 生成输入场（所有模型使用相同的输入场）
    print("生成输入场...")
    input_field = data_generator.generate_input_data()
    
    # 为每个有效模型进行仿真
    for model_idx, (idx, model, visibility, num_layers) in enumerate(valid_models):
        print(f"\n--- 处理第 {model_idx+1}/{len(valid_models)} 个模型: {num_layers}层 ---")
        print(f"平均可见度: {np.mean(visibility):.4f}")
        
        # 获取该模型的相位掩码
        phase_masks = results['phase_masks'][idx]
        
        # 获取仿真掩码
        simulation_masks = mask_loader.get_masks_for_simulation(
            trained_masks=phase_masks, 
            num_layers=num_layers
        )
        
        try:
            # 创建模拟器
            print(f"创建 {num_layers} 层模型的模拟器...")
            simulator = Simulator(config, evaluation_regions=evaluation_regions)
            
            # 为每个模式生成专用相位掩膜
            print(f"为 {num_layers} 层模型生成专用相位掩膜...")
            mode_specific_masks = simulator.generate_mode_specific_masks(simulation_masks, config.num_modes)
            
            # 模拟光场传播
            print(f"模拟 {num_layers} 层模型的光场传播...")
            simulator.simulate_propagation(
                simulation_masks, 
                input_field, 
                process_all_modes=True,
                mode_specific_masks=mode_specific_masks
            )
            
            # 打印相位掩膜信息
            print(f"保存 {num_layers} 层模型的相位掩膜信息...")
            if hasattr(model, 'print_phase_masks'):
                # 为每个模型创建单独的保存目录
                model_save_dir = os.path.join(config.save_dir, f"{num_layers}layer_model")
                os.makedirs(model_save_dir, exist_ok=True)
                model.print_phase_masks(save_path=model_save_dir)
            
            print(f"✓ {num_layers} 层模型仿真完成")
            
        except Exception as e:
            print(f"❌ {num_layers} 层模型仿真过程出错: {e}")
            print("继续处理下一个模型...")
            continue
    
    # 生成所有模型的性能对比总结
    print("\n--- 所有模型仿真结果总结 ---")
    print("模型性能对比:")
    
    for model_idx, (idx, model, visibility, num_layers) in enumerate(valid_models):
        avg_visibility = np.mean(visibility)
        print(f"  {num_layers}层模型: 平均可见度 = {avg_visibility:.4f}")
        
        # 显示各波长各模式的详细可见度
        vis_array = np.array(visibility).reshape(len(config.wavelengths), config.num_modes)
        for wl_idx, wavelength in enumerate(config.wavelengths):
            wl_nm = wavelength * 1e9
            modes_vis = vis_array[wl_idx, :]
            mode_str = ", ".join([f"模式{i+1}={v:.3f}" for i, v in enumerate(modes_vis)])
            print(f"    {wl_nm:.0f}nm: {mode_str}")
    
    # 找出最佳模型
    best_model_info = max(valid_models, key=lambda x: np.mean(x[2]))
    best_idx, best_model, best_visibility, best_num_layers = best_model_info
    
    print(f"\n🏆 最佳模型: {best_num_layers}层")
    print(f"   平均可见度: {np.mean(best_visibility):.4f}")
    
    print(f"\n✅ 所有 {len(valid_models)} 个模型的仿真完成")
    
else:
    print("❌ 没有有效的训练结果，跳过仿真")

print("✓ 光场传播仿真阶段完成")


In [ ]:

print("✓ 所有结果保存完成")

# ===== 程序完成 =====
print("\n" + "="*60)
print("程序执行完成!")
print("="*60)

# 计算总执行时间
total_end_time = time.time()
total_time = total_end_time - start_time
print(f"总执行时间: {total_time:.2f} 秒 ({total_time/60:.2f} 分钟)")

# 打印最终摘要
print("\n=== 执行摘要 ===")
print(f"训练层数选项: {num_layer_options}")
print(f"保存目录: {config.save_dir}")
if 'best_idx' in locals():
    print(f"最佳模型: {best_num_layers}层 (平均可见度: {np.mean(best_visibility):.4f})")
print(f"结果文件: complete_results_{timestamp}.pth")
print("="*40)